## NB: python3-only

In [1]:
from zipfile import ZipFile
from io import BytesIO
import urllib.request

ZipFile.extract(
    ZipFile(
        BytesIO(
            urllib
            .request
            .urlopen('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip')
            .read()
        )
    ),
    'top-1m.csv'
)

'/Users/badanin/Downloads/top-1m.csv'

# Дз №1. Мапредьюс руками

Напишите маппер и редьюсер для задачи обратного индекса по рейтингу сайтов алексы.
Задача построения обратного индекса означает, что нужно разбить все домены в рейтинге на части(токены) по символу точки, а на выходе получить пары (токен, все сайты в названии которых содержался токен). Пример:

```
(
    google.com,
    google.ru,
    ya.ru,
    hse.ru
)

```
должно на выходе превратиться в:

```python
(
    (com, (google.com,)),
    (google, (google.com, google.ru)),
    (ru, (google.ru, ya.ru, hse.ru)),
    (ya, (ya.ru, )),
    (hse, (hse.ru, ))
)
```


In [2]:
with open('top-1m.csv') as alexa:
    alexa_domains = [s.strip().split(",")[1] for s in alexa]

## 1.1 Маппер

In [ ]:
def mapper(domain):
    """
    Маппер для обратного индекса. Разбивает доменное имя на токены и помечает каждый токен доменом. 
    Пример: google.com -> ('com', 'google.com'), ('google', 'google.com')
    """
    # Ваш код здесь

## 1.2 Редьюсер

In [ ]:
def reducer(token_domain_pairs):
    """
    Редьюсер для обратного индекса. По набору кортежей, созданных маппером, возвращает кортеж 
    вида (токен, (домен1, домен2, ...)
    Пример:
    (('google', 'google.com'), ('google', 'google.se')) -> ('google', ('google.com', 'google.se'))
    Примечание: гарантируется, что у всех кортежей на входе оданковый токен.
    """
    # Ваш код здесь

    

## bonus:

Напишите также несколько тестов, чтобы проверить корректность маппера и редьюсера

например таких:

In [5]:
assert set(mapper("ya.ru")) == {("ya", "ya.ru"), ("ru", "ya.ru")}
assert set(mapper("com")) == {("com", "com")}

def reducer_test(testcase, expected_result):
    actual_result = reducer(testcase)
    assert actual_result[0] == expected_result[0]
    assert set(actual_result[1]) == set(expected_result[1])
    
reducer_test(
    (('google', 'google.com'), ('google', 'google.se')),
    ('google', ('google.com', 'google.se'))
)

In [6]:
def flatmap(mapper_func, iterable):
    from itertools import chain
    return tuple(chain.from_iterable(map(mapper_func, iterable)))

#### проверим как работает ваш маппер на датасете:

In [7]:
flatmap(mapper, alexa_domains[:10])

(('google', 'google.com'),
 ('com', 'google.com'),
 ('youtube', 'youtube.com'),
 ('com', 'youtube.com'),
 ('facebook', 'facebook.com'),
 ('com', 'facebook.com'),
 ('baidu', 'baidu.com'),
 ('com', 'baidu.com'),
 ('wikipedia', 'wikipedia.org'),
 ('org', 'wikipedia.org'),
 ('qq', 'qq.com'),
 ('com', 'qq.com'),
 ('tmall', 'tmall.com'),
 ('com', 'tmall.com'),
 ('taobao', 'taobao.com'),
 ('com', 'taobao.com'),
 ('yahoo', 'yahoo.com'),
 ('com', 'yahoo.com'),
 ('amazon', 'amazon.com'),
 ('com', 'amazon.com'))

## 1.3 Шаффл
напишите функцию, которая сгруппирует всю выдачу всех мапперов по токену, так чтобы соблюдалось предположение, принятое нами в определении редьюсера:

> `Примечание: гарантируется, что у всех кортежей на входе оданковый токен.`

#### Примечание: в идеале, в реализации шафла не пользоваться питоньими диктами (`{k:v}` и т.д )

In [ ]:
def shuffle(dataset):
    """
    Группирует с выводом мапперов по ключу. Пример:
    
    (
        ('google', 'google.com'),
        ('com', 'google.com'),
        ('youtube', 'youtube.com'),
        ('com', 'youtube.com')
    )

    --> shuffle все превращает в --> 

    (
        [('com', 'google.com'), ('com', 'youtube.com')],
        [('google', 'google.com')],
        [('youtube', 'youtube.com')],
    )

    :param dataset:
    :return:
    
    """
    # Ваш код здесь

### Собираем все вместе

In [9]:
mapper_output = flatmap(mapper, alexa_domains)
shuffled = shuffle(mapper_output)
reduced = map(reducer, shuffled)

reversed_idx = dict(reduced)

Должно посчитаться секунд за пять. Если не получается и считается долго, то прежде чем оптимизировать код попробуйте подставить небольшое подмножество `alexa_domains`, например `alexa_domains[:100]`. Корректность кода важнее быстродействия.

`reversed_idx` должен содержать то что нужно для поиска по токену:

In [10]:
reversed_idx['yandex']

('yandex.ee',
 'yandex.uk.com',
 'yandex.kz',
 'yandex.kg',
 'yandex.net',
 'yandex.tm',
 'yandex.lv',
 'yandex.lt',
 'clickhouse.yandex',
 'yandex.ru',
 'yandex.uz',
 'yandex.az',
 'yandex.tj',
 'driver.yandex',
 'auto.yandex',
 'yandex.com.ge',
 'yandex.com',
 'yandex.co.il',
 'yandex.com.tr',
 'eda.yandex',
 'yandex.fr',
 'yandex.md',
 'ir.yandex',
 'yandex.org.kz',
 'yandex.by',
 'yandex.ua')